In [1]:
import numpy as np
import pandas as pd
import requests 
import math
from scipy import stats
import xlsxwriter

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
from API_token import IEX_CLOUD_API_TOKEN
def chunks(lst, n):
    # Yield successive n-sized chungs from list
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        
# IEX Cloud limits batch API calls to 100 tickers per request to boost up the time 
# Splitting into groups of 100 tickers
symbol_groups = list(chunks(stocks["Ticker"], 100))

In [19]:
# Information from the API
# Market cap for each stock and price of each stock

symbol_strings = []
# creating strings of symbol for batch API calls
for i in range(len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

columns = ["Ticker", "Price", "beta", "6 Month Return", "1 Year Return", "Number of Shares to Buy"]
finalDF = pd.DataFrame(columns=columns)
# parsing the data to input into the final dataframe
for symbols in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call).json()
    for symbol in symbols.split(","):
        finalDF = finalDF.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["price"],
                    data[symbol]["stats"]["beta"],
                    data[symbol]["stats"]["month6ChangePercent"],
                    data[symbol]["stats"]["ytdChangePercent"],
                    "N/A"
                ],
            index = columns
            ),
            ignore_index = True
        )
        
        

In [20]:
finalDF

,Ticker,Price,beta,6 Month Return,1 Year Return,Number of Shares to Buy
0,A,150.26,1.02383,-0.0247628,-0.0947745,N/A
1,AAL,18.68,1.06636,-0.105779,0.0309601,N/A
2,AAP,241.90,0.788454,0.157657,-0.00118233,N/A
3,AAPL,173.34,1.35078,0.169958,-0.0255784,N/A
4,ABBV,141.89,0.599779,0.1862,0.0145081,N/A
...,...,...,...,...,...,...
499,YUM,127.77,0.675196,0.0960269,-0.0856103,N/A
500,ZBH,130.76,0.882184,-0.182102,0.00272995,N/A
501,ZBRA,534.00,1.39716,-0.00421564,-0.11701,N/A
502,ZION,72.02,1.25062,0.396343,0.120974,N/A


In [22]:
# investment strategy we building seeks to indentify the 50 highest-momentum or highest return stocks in S&P 500 
finalDF.sort_values("6 Month Return", ascending=False, inplace=True)
finalDF = finalDF[:50]
finalDF.reset_index(inplace = True)

def portfolio_input():
    global portfolio_size
    
    while True:
        # Enter integer value from the console.
        portfolio_size = input('Enter the value of your portfolio: ')

        try: 
            float(portfolio_size)
        except ValueError:
            print("Please enter a number")
        else:
            break
            
portfolio_input()        
portfolio_size = float(portfolio_size)/len(finalDF.index)
for i in range(len(finalDF.index)):
    finalDF.loc[i, "Number of Shares to Buy"] = portfolio_size // finalDF.loc[i, "Price"]
    
finalDF

<ipython-input-22-a0d10ace9ca4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDF.sort_values("6 Month Return", ascending=False, inplace=True)


,index,Ticker,Price,beta,6 Month Return,1 Year Return,Number of Shares to Buy
0,299,MCHP,86.12,1.93866,1.41792,-0.0332269,N/A
1,148,DVN,51.79,1.63202,0.93379,0.149315,N/A
2,275,LB,81.00,1.37712,0.826011,1.20051,N/A
3,174,F,25.69,1.24066,0.791272,0.214277,N/A
4,42,APA,34.33,0,0.77015,0.245133,N/A
5,317,MRO,20.28,1.59197,0.593748,0.189092,N/A
6,175,FANG,134.79,1.67091,0.552672,0.202517,N/A
7,109,COP,90.86,1.11823,0.548926,0.206914,N/A
8,31,AMD,139.25,1.78238,0.541192,-0.0495313,N/A
9,98,CMA,103.24,1.1998,0.50945,0.165072,N/A


In [29]:
# A more realistic and better momentum strategy
# High quality momentum stocks show "SLOW AND STEADY" outperformance over long periods of time
    # Building a strategy select stocks from the highest percentile of 1-month, 3-month, 6-month, 1-year returns
    
# Low quality momentum stocks migh not show any momentum for a long time, and then suddenly surge upwards
    # ~2-3% returns for 11 months then ~10-15% in one month 
    # events like FDA approval is the reason

In [142]:
hqm_colunm = [
    "Ticker",
    "Price",
    "Number of Shares to Buy",
    "1-Year Return",
    "1-Year Return Percentile",
    "6-month Return",
    "6-month Return Percentile",
    "3-month Return",
    "3-month Return Percentile",
    "1-month Return",
    "1-month Return Percentile",
    "HQM Score"
]
hqm_df = pd.DataFrame(columns=hqm_colunm)

for symbols in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=price,stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call).json()
    for symbol in symbols.split(","):
        hqm_df = hqm_df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]["price"],
                    "N/A",
                    data[symbol]["stats"]["year1ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month6ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month3ChangePercent"],
                    "N/A",
                    data[symbol]["stats"]["month1ChangePercent"],
                    "N/A",
                    "N/A"
                    
                ],
            index = hqm_colunm
            ),
            ignore_index = True
        )

# Dropping None returns stocks
hqm_df.drop(list(hqm_df[hqm_df['3-month Return'].isnull()].index), inplace=True)

time_periods = [
    "1-Year",
    "6-month",
    "3-month",
    "1-month"
]

for row in hqm_df.index:
    momentum_percentile = []
    for time_period in time_periods:
        returnPercentile = f"{time_period} Return Percentile"
        percentChange = f"{time_period} Return"
        hqm_df.loc[row, returnPercentile] = stats.percentileofscore(hqm_df[percentChange], hqm_df.loc[row, percentChange]) / 100
        momentum_percentile.append(hqm_df.loc[row, returnPercentile])
        
    # High Quality Momentum Score is the arithmetic mean of the 4 momentum percentile scores
    hqm_df.loc[row, "HQM Score"] = np.array(momentum_percentile).mean()
        
        
# top 50 hqm stocks
hqm_df.sort_values("HQM Score", ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop = True, inplace=True)
portfolio_input()
position_size = float(portfolio_size)/len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i, "Number of Shares to Buy"] = position_size//hqm_df.loc[i, "Price"]
hqm_df



Enter the value of your portfolio: 10000000


,Ticker,Price,Number of Shares to Buy,1-Year Return,1-Year Return Percentile,6-month Return,6-month Return Percentile,3-month Return,3-month Return Percentile,1-month Return,1-month Return Percentile,HQM Score
0,F,25.37,7883,1.52655,0.996,0.790854,0.994,0.639369,1,0.258883,0.984,0.9935
1,DVN,52.08,3840,1.6132,0.998,0.945482,0.998,0.304656,0.98,0.257544,0.98,0.989
2,APA,34.11,5863,0.717012,0.97,0.781447,0.992,0.282878,0.97,0.345862,0.996,0.982
3,MRO,20.11,9945,1.30928,0.994,0.586925,0.99,0.220349,0.928,0.284091,0.99,0.9755
4,FANG,133.39,1499,1.16746,0.99,0.557756,0.986,0.19688,0.902,0.243364,0.966,0.961
5,EOG,106.63,1875,0.814147,0.978,0.398459,0.964,0.202773,0.914,0.256763,0.978,0.9585
6,CMA,104.31,1917,0.613262,0.934,0.518936,0.982,0.242328,0.94,0.215019,0.954,0.9525
7,COP,91.02,2197,0.909309,0.98,0.535636,0.984,0.187868,0.886,0.23832,0.96,0.9525
8,WFC,60.54,3303,0.697866,0.96,0.305439,0.92,0.293973,0.972,0.196233,0.936,0.947
9,SCHW,97.24,2056,0.636825,0.946,0.376772,0.956,0.22703,0.936,0.184959,0.928,0.9415


In [143]:
# Writing to excel
writer = pd.ExcelWriter("RecommendedTrades.xlsx", engine="xlsxwriter")
hqm_df.to_excel(writer, "RecommendedTrades", index = False)

# setting background and font color
background_color = "#0a0a23"
font_color = "#ffffff"

# String format for ticker
# $XX.XX format for stock price
# $XX,XXX format for market cap
# integer format for the number of shares purchase
string_format = writer.book.add_format(
    {
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

integer_format = writer.book.add_format(
    {
        "num_format": "0.0",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

percent_format = writer.book.add_format(
    {
        "num_format": "0.0%",
        "font_color": font_color,
        "bg_color": background_color,
        "border": 1
    }
)

column_format = {
    "A": ["Ticker", string_format],
    "B": ["Price", dollar_format],
    "C": ["Number of Shares to Buy", integer_format],
    "D": ["1-Year Return", percent_format],
    "E": ["1-Year Return Percentile", percent_format],
    "F": ["6-month Return", percent_format],
    "G": ["6-month Return Percentile", percent_format],
    "H": ["3-month Return", percent_format],
    "I": ["3-month Return Percentile", percent_format],
    "J": ["1-month Return", percent_format],
    "K": ["1-month Return Percentile", percent_format],
    "L": ["HQM Score", percent_format]
}

# Formatting each column to its readable form 
# and reformat the column headers    
for column in column_format.keys():
    writer.sheets["RecommendedTrades"].set_column(f"{column}:{column}", 18, column_format[column][1]) # 18 is the pixel for the size of the column
    writer.sheets["RecommendedTrades"].write(f"{column}1", column_format[column][0], column_format[column][1])
    
writer.save()


In [22]:
from API_token import IEX_CLOUD_API_TOKEN
symbol = "FB"
indicator_name = "sma"
batch_api_call = f"https://sandbox.iexapis.com/stable/stock/{symbol}/indicator/{indicator_name}?range=6&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call).json()

In [23]:
data

{'indicator': [[None]],
 'chart': [{'close': 343.4,
   'high': 343.4,
   'low': 321.41,
   'open': 329.18,
   'symbol': 'FB',
   'volume': 17057184,
   'id': 'SIRCH_PRIIECLTOSA',
   'key': 'BF',
   'subkey': '',
   'date': '2022-01-14',
   'updated': 1656749562904,
   'changeOverTime': 0,
   'marketChangeOverTime': 0,
   'uOpen': 337.59,
   'uClose': 345.9,
   'uHigh': 338.82,
   'uLow': 336.75,
   'uVolume': 17209980,
   'fOpen': 329.79,
   'fClose': 342.8,
   'fHigh': 346.28,
   'fLow': 331.24,
   'fVolume': 16999546,
   'label': 'Jan 14, 22',
   'change': 0,
   'changePercent': 0}]}